In [2]:
pip install pyPDF2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 KB 16.8 MB/s eta 0:00:00


In [3]:
from PyPDF2 import PdfReader

In [4]:
!pip install textract

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [6]:
!pip install regex

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [7]:
import pandas as pd
import numpy as np
import PyPDF2
import textract
import re

In [22]:
filename ="/content/sample_data/Resume.pdf" 

pdfFileObj = open(filename,'rb')               #open allows you to read the file
pdfReader = PyPDF2.PdfReader(pdfFileObj)   #The pdfReader variable is a readable object that will be parsed
num_pages = len(pdfReader.pages)                #discerning the number of pages will allow us to parse through all the pages


count = 0
text = ""
                                                            
while count < num_pages:                       #The while loop will read each page
    pageObj = pdfReader.pages[count]
    count +=1
    text += pageObj.extract_text()
    
#Below if statement exists to check if the above library returned #words. It's done because PyPDF2 cannot read scanned files.

if text != "":
    text = text
    
#If the above returns as False, we run the OCR library textract to #convert scanned/image based PDF files into text

else:
    text = textract.process('http://bit.ly/epo_keyword_extraction_document', method='tesseract', language='eng')

    # Now we have a text variable which contains all the text derived from our PDF file.

In [23]:
text = text.encode('ascii','ignore').lower() #Lowercasing each word

In [31]:
#keywords = re.findall(r"[a-zA-Z]\w+",text)
keywords = re.findall(b"[A-Za-z]",text)
len(keywords) 

1434

In [32]:
df = pd.DataFrame(list(set(keywords)),columns=['keywords'])

In [33]:
def weightage(word,text,number_of_documents=1):
    word_list = re.findall(word,text)
    number_of_times_word_appeared =len(word_list)
    tf = number_of_times_word_appeared/float(len(text))
    idf = np.log((number_of_documents)/float(number_of_times_word_appeared))
    tf_idf = tf*idf
    return number_of_times_word_appeared,tf,idf ,tf_idf

In [34]:
df['number_of_times_word_appeared'] = df['keywords'].apply(lambda x: weightage(x,text)[0])
df['tf'] = df['keywords'].apply(lambda x: weightage(x,text)[1])
df['idf'] = df['keywords'].apply(lambda x: weightage(x,text)[2])
df['tf_idf'] = df['keywords'].apply(lambda x: weightage(x,text)[3])

In [35]:
df = df.sort_values('tf_idf',ascending=True)
df.to_csv('Keywords.csv')
df.head(25)

,keywords,number_of_times_word_appeared,tf,idf,tf_idf
2,b'a',147,0.076602,-4.990433,-0.382279
8,b'e',145,0.075560,-4.976734,-0.376043
18,b'i',132,0.068786,-4.882802,-0.335868
7,b't',128,0.066701,-4.852030,-0.323637
3,b'n',125,0.065138,-4.828314,-0.314507
0,b's',111,0.057843,-4.709530,-0.272412
10,b'r',89,0.046378,-4.488636,-0.208175
6,b'o',86,0.044815,-4.454347,-0.199622
11,b'd',57,0.029703,-4.043051,-0.120091
14,b'c',54,0.028140,-3.988984,-0.112249
